finding cat_cols:

In [ ]:
cat_cols=df.select_dtypes("object").columns

scaling data:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

scremating features:

In [ ]:
from sklearn.feature_selection import RFECV

selector = RFECV(logreg, cv=5) 
selector = selector.fit(X, y) 
  
# Print the optimal number of features 
print("Optimal number of features: %d" % selector.n_features_) 
  
# Print the selected features 
print("Selected features: %s" % selector.support_) 

X=X[selector.get_feature_names_out()]

train-test-split:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40, shuffle=True)

grid search:

In [ ]:

grid_search = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = "f1",
                           cv = StratifiedKFold(5),
                           verbose=0,
                           )

grid_search.fit(X_train, y_train)   

best_model=grid_search.best_estimator_
best_model.fit(X_train, y_train)

confusion matrix:

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt

conf_matrix=confusion_matrix(y_test, best_model.predict(X_test))

cm_display = ConfusionMatrixDisplay(confusion_matrix = conf_matrix, display_labels = [0, 1])

cm_display.plot()
plt.show() 

classification report:

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, best_model.predict(X_test)))

f1 score:

In [ ]:
from sklearn.metrics import make_scorer,f1_score
f1_score(y_true=y_test,y_pred=best_model.predict(X_test))

pipeline es:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

numeric_features=list(X.select_dtypes(exclude='object'))
categorical_features=list(X.select_dtypes(include='object'))

pipeline = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numeric_features),
            ('cat', SimpleImputer(strategy='mode', fill_value='missing'), 
             categorical_features)
        ],
        remainder='drop'  # This will drop any column not specified in the transformers
    )),
    ('classifier', CatBoostClassifier(
        iterations=300,
        learning_rate=0.1,
        depth=6,
        verbose=100,
        cat_features=list(range(len(numeric_features), 
                              len(numeric_features) + len(categorical_features)))
    ))
])

pipeline.fit(X_train,y_train)

import joblib

joblib.dump(pipeline,"pipe.pkl")

shap tree explainer:

In [ ]:
explainer = shap.TreeExplainer(best_model)
# Take first 1000 samples for computation efficiency
shap_values = explainer.shap_values(X_test.iloc[:1000])

# Summary plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_test.iloc[:1000], plot_type="bar")
plt.title("SHAP Feature Importance")
plt.tight_layout()
plt.show()

roc curve:

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Get prediction probabilities
y_pred_proba = best_model.predict_proba(X_test)[:,1]

# Calculate ROC curve and thresholds
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10,6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')

# Add threshold annotations
for i in range(0, len(thresholds), len(thresholds)//5):  # Plot every nth threshold
   plt.annotate(f'{thresholds[i]:.2f}', 
               (fpr[i], tpr[i]),
               xytext=(10, 10),
               textcoords='offset points')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with Thresholds')
plt.legend()
plt.grid(True)
plt.show()

remove columns limit:

In [ ]:
pd.set_option( "display.max_columns", None)

shuffle dataframe

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)

xgboost param grid:

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1]
}

logreg param grid:

In [ ]:
parameters = {
            #'solver':['liblinear',"lbfgs"], 
            #{'C':[0.01,0.1,1, 10, 100]}
            #'class_weight':[{1:2,0:1}]
            }

random forest grid:

In [ ]:
param_grid = {
    'n_estimators': [200, 700],
    'max_features': ['auto', 'sqrt', 'log2']
}

decision tree classifier

In [ ]:
param_grid = {
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

stacking:

In [ ]:
base_models = [
    ('KNN', KNeighborsRegressor()),
    ('SVR',SVR()),
    ('Random Forest',RandomForestRegressor()),
    ('Linear Regression',LinearRegression()),
    ]

stacked = StackingRegressor(
    estimators = base_models,
    final_estimator = LinearRegression(),
    cv = 5)